# MELTS v 1.0.2 Implemented Using the `Equilibrate` Class
The `Equilibrate` class generates an equilibrium assemblage when provided with a set of phases, a bulk composition, and <strong>T,P</strong> or <strong>S,P</strong> or <strong>T,V</strong> or <strong>S,V</strong>.  
  
Versions of MELTS implemented are:  
- MELTS v. 1.0.2 ➞ [EquilibrateUsingMELTSv102](https://gitlab.com/ENKI-portal/PhaseOb→jC/blob/master/Documentation/EquilibrateUsingMELTSv102.md) (rhyolite-MELTS)  
- MELTS v. 1.1.0 ➞ EquilibrateUsingMELTSv110 (rhyolite-MELTS + new CO<sub>2</sub>, works at the ternary minimum)  
- MELTS v. 1.2.0 ➞ EquilibrateUsingMELTSv120 (rhyolite-MELTS + new H<sub>2</sub>O + new CO<sub>2</sub>)  
- pMELTS v. 5.6.1 ➞ EquilibrateUsingpMELTSv561  

## Initialize tools and package that are required to execute this notebook.

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))
import matplotlib.pyplot as plt
import numpy as np
import ctypes
%matplotlib inline

## Create an instance of the `EquilibrateUsingMELTSv102` subclass of `Equilibrate`.
This class implements the phase model collection corresponding to rhyolite-MELTS v 1.0.2 (Gualda et al., 2012). 

In [ ]:
EquilibrateUsingMELTSv102 = ObjCClass('EquilibrateUsingMELTSv102')
melts = EquilibrateUsingMELTSv102.alloc().init()
melts.setDebugS_(0)
melts.setDebugV_(0)

### Optionally, generate some information about the implemented model.
Set <strong>False</strong> to <strong>True</strong> in the following lines of code in order to display a list of system oxide components known to MELTS v 1.0.2 and a list of known liquid, solid, and fluid phases.

In [ ]:
oxides = melts.oxideNames()
if False:
    print ('System oxide components:')
    for i in range (0,oxides.count):
        print ('   ', oxides.objectAtIndex_(i))
phases = melts.phaseNames()
if False:
    print ('Known liquid, solid and fluid phases:')
    for i in range (0,phases.count):
        print ('   ', phases.objectAtIndex_(i))

## Configure an equilibrium crystallization calculation.
### We input initial composition of the system (liquid). 
The first two lines of code are required in order to produce a "C"-style array of double precision values that are acceptable as input to the class method.

In [ ]:
wt = (ctypes.c_double*oxides.count)()
ctypes.cast(wt, ctypes.POINTER(ctypes.c_double))
wt[0] = 77.8   # SiO2
wt[1] =  0.09  # TiO2
wt[2] = 12.0   # Al2O3
wt[3] =  0.196 # Fe2O3
wt[4] =  0.0   # Cr2O3
wt[5] =  0.474 # FeO
wt[6] =  0.0   # MnO
wt[7] =  0.04  # MgO
wt[8] =  0.0   # NiO
wt[9] =  0.0   # CoO
wt[10] = 0.45  # CaO
wt[11] = 3.7   # Na2O
wt[12] = 5.36  # K2O
wt[13] = 0.0   # P2O5
wt[14] = 3.74  # H2O
melts.setComposition_(wt)

### Input initial temperature and pressure.

In [ ]:
t = 770.0 + 273.15 # K
p = 1750.0 # bars
melts.setTemperature_(t)
melts.setPressure_(p)

## Compute the equilibrium state at the initial conditions.


In [ ]:
output = melts.execute()
print (output.objectForKey_('status'), t-273.15, p)

## There are a number of ways to process output:
### Method 1 
The following functions process results stored directly by data structures returned by the `Equilibrate` class. The execute() method of the class returns a two-member dictionary, with keys status and results  
- <strong>status</strong> points to a string that describes the manner in which the method exited.  
- <strong>results</strong> contains the results of the calculation. It is an instance of the `EquilibrateState` class.  
- <strong>results</strong> has a property called `phasesInSystem`, which is a dictionary of phases in the equilibrium assemblage, with keys given by phase names. Each object in the dictionary is an instance of the class `EquilibrateStatePhase`. Each `EquilibrateStatePhase` object exposes properties about the phase, including mass, affinity, bulk composition, and a pointer to the instance of the class that implements the phase thermodynamics.  

Consult the documentation on the three classes—`Equilibrate`, `EquilibrateState`, and `EquilibrateStatePhase`—for more information.  
  
<> <strong>processOutput</strong> returns output generated by the `execute` command as a Python dictionary, and optionally prints a summary of results.

In [ ]:
e = (ctypes.c_double*107)()
ctypes.cast(e, ctypes.POINTER(ctypes.c_double))
m = (ctypes.c_double*15)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))

def processOutput(generateOutput=False):
    results = output.objectForKey_('results') # EquilibrateState class instance
    phases = results.phasesInSystem.allKeys() # array of strings, one for each phase in the assemblage
    if (generateOutput):
        print ('Temperature (°C)', results.T-273.15)
    phaseDict = {}
    for i in range(0, phases.count):
        phase = phases.objectAtIndex_(i) # name of phase
        obj = results.phasesInSystem.objectForKey_(phase) # EquilibrateStatePhase class instance
        if (generateOutput):
            print ("Mass of ", phase, " in equilibrium assemblage is ", obj.mass, " grams")
        phaseDict[phase] = {}
        phaseDict[phase]['mass'] = obj.mass
        phaseInstance = obj.phaseClassInstance
        bulkComposition = obj.bulkCompositionInElements
        for j in range (0, 107):
            e[j] = bulkComposition.valueAtIndex_(j)
        try:
            moles = phaseInstance.convertElementsToMoles_(e)
            nc = phaseInstance.numberOfSolutionComponents()
            for j in range (0, nc):
                m[j] = moles.valueAtIndex_(j)
            if (generateOutput):
                print ('     formula = ', phaseInstance.getFormulaFromMolesOfComponents_andT_andP_(m, t, p))
        except AttributeError:
            if (generateOutput):
                print ('     formula = ', phaseInstance.phaseFormula)
    return phaseDict

#### Using Method 1 - Output results to a Python dictionary and the screen.

In [ ]:

phaseDict = processOutput(generateOutput=True)

### Method 2:  Alternative output using XML retrieved from `EquilibrateState` object
This method using an XML tree-like data structure that is generated by the equilibrate class that yields the most comprehensive set of outputs

In [ ]:
import xml.etree.ElementTree as ET
root = ET.fromstring(melts.equilibrateResultsAsXML())

print ("T (°C)", root.find(".//Temperature").text)
print ("P (MPa)", float(root.find(".//Pressure").text)*1000.0)
print ("Mass (g)", root.find(".//Mass").text)

print ("Bulk composition in elemental abundances (moles):")
bcElements = list(root.findall(".//Composition/Element"))
for element in bcElements:
    print ("   ", element.attrib['Type'], element.text)

print ("Bulk composition in oxide abundances (wt %):")
bcOxides = list(root.findall(".//Composition/Oxide"))
for oxide in bcOxides:
    print ("   ", oxide.attrib['Type'], oxide.text)
    
phases = list(root.findall(".//System/Phase"))
for phase in phases:
    print (phase.attrib['Type'])
    print ("   Mass (g)", phase.find("Mass").text)
    print ("   Formula", phase.find("Formula").text)
    oxides = list(phase.findall("Oxide"))
    for oxide in oxides:
        value = float(oxide.text)
        if (value != 0.0):
            print ("   ", oxide.attrib['Type'], oxide.text)
    components = list(phase.findall("Component"))
    for component in components:
        value = float(component.text)
        if (value != 0.0):
            print ("   ", component.attrib['Name'], component.text)

### Method 3: Define some functions to output results into an Excel notebook
Utilize the `openpyxl` Python package to read/write Excel notebooks.

In [ ]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter

def startExcelWorkbookWithSheetName(sheetName="Summary"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheetName
    return wb

def addSheetToWorkbookNamed(wb, sheetName):
    ws = wb.create_sheet(title=sheetName)
    return ws

def writeToCellInSheet(ws, col, row, value, format='general'):
    if format == 'number':
        ws.cell(column=col, row=row, value=float(value)).number_format = '0.00'
    elif format == 'scientific':
        ws.cell(column=col, row=row, value=float(value)).number_format = '0.00E+00'
    else:
        ws.cell(column=col, row=row, value=value)

def writeExcelWorkbook(wb, fileName="junk.xlsx"):
    wb.save(filename = fileName)

# row is an externally defined variable
def updateExcelWorkbook(wb, root):
    global row
    
    t = root.find(".//Temperature").text
    p = float(root.find(".//Pressure").text)*1000.0
    bcElements = list(root.findall(".//Composition/Element"))
    bcOxides = list(root.findall(".//Composition/Oxide"))
    
    wsSummary = wb.get_sheet_by_name("Summary")
    if (row == 0):
        col = 1
        row = 1
        writeToCellInSheet(wsSummary, col, row, "T °C")
        col += 1
        writeToCellInSheet(wsSummary, col, row, "P MPa")
        col += 1
        writeToCellInSheet(wsSummary, col, row, "Mass g")
        col += 1
        for element in bcElements:
            writeToCellInSheet(wsSummary, col, row, element.attrib['Type'])
            col += 1
        for oxide in bcOxides:
            writeToCellInSheet(wsSummary, col, row, oxide.attrib['Type'])
            col += 1
    
    row += 1
    col = 1
    writeToCellInSheet(wsSummary, col, row, t, format='number')
    col += 1
    writeToCellInSheet(wsSummary, col, row, p, format='number')
    col += 1
    writeToCellInSheet(wsSummary, col, row, root.find(".//Mass").text, format='number')
    col += 1
    for element in bcElements:
        writeToCellInSheet(wsSummary, col, row, element.text, format='scientific')
        col += 1
    for oxide in bcOxides:
        writeToCellInSheet(wsSummary, col, row, oxide.text, format='number')
        col += 1
    
    phases = list(root.findall(".//System/Phase"))
    for phase in phases:
        phaseType = phase.attrib['Type']
        oxides = list(phase.findall("Oxide"))
        components = list(phase.findall("Component"))
        
        try:
            wsPhase = wb.get_sheet_by_name(phaseType)
        except KeyError:
            wsPhase = wb.create_sheet(phaseType)
            col = 1
            writeToCellInSheet(wsPhase, col, 1, "T °C")
            col += 1
            writeToCellInSheet(wsPhase, col, 1, "P MPa")
            col += 1
            writeToCellInSheet(wsPhase, col, 1, "Mass g")
            col += 1
            writeToCellInSheet(wsPhase, col, 1, "Formula")
            col += 1
            for oxide in oxides:
                writeToCellInSheet(wsPhase, col, 1, oxide.attrib['Type'])
                col += 1
            for component in components:
                writeToCellInSheet(wsPhase, col, 1, component.attrib['Name'])
                col += 1
        
        col = 1
        writeToCellInSheet(wsPhase, col, row, t, format='number')
        col += 1
        writeToCellInSheet(wsPhase, col, row, p, format='number')
        col += 1
        writeToCellInSheet(wsPhase, col, row, phase.find("Mass").text, format='number')
        col += 1
        writeToCellInSheet(wsPhase, col, row, phase.find("Formula").text)
        col += 1
        for oxide in oxides:
            writeToCellInSheet(wsPhase, col, row, oxide.text, format='number')
            col += 1
        for component in components:
            writeToCellInSheet(wsPhase, col, row, component.text, format='scientific')
            col += 1


## Now, use Method 3 to output results from a sequence of calculations . . . 
... and, while you are at it, plot the results. 

Choose temperature increment, number of steps, phases to plot, and color/style for plot display:  
b : blue.  
g : green.  
r : red.  
c : cyan.  
m : magenta.  
y : yellow.  
k : black.  
w : white.  

In [ ]:
tMin = 750.0
tMax = 770.0
tIncrement = -1.0
increments = 20
plotPhases = ['Liquid', 'Sanidine', 'Plagioclase', 'Quartz']
plotColors = [ 'ro', 'bo', 'go', 'co']

## Run the sequence of calculations.

In [ ]:
wb = startExcelWorkbookWithSheetName(sheetName="Summary")
row = 0

n = len(plotPhases)
xPlot = np.zeros(increments+1)
yPlot = np.zeros((n, increments+1))
xPlot[0] = t - 273.15
for i in range (0, n):
    if root.find(".//System/Phase[@Type='" + plotPhases[i] + "']/Mass") != None:
        value = root.find(".//System/Phase[@Type='" + plotPhases[i] + "']/Mass").text
    else:
        value = 0.0
    yPlot[i][0] = value
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([tMin, tMax])
ax.set_ylim([0., 100.])
graphs = []
for i in range (0, n):
    graphs.append(ax.plot(xPlot, yPlot[i], plotColors[i]))
handle = []
for (graph,) in graphs:
    handle.append(graph)
ax.legend(handle, plotPhases, loc='upper left')
updateExcelWorkbook(wb, root)

for i in range (1, increments):
    t = t + tIncrement
    melts.incrementTemperature_(t)
    output = melts.execute()
    print (output.objectForKey_('status'), t-273.15, p)
    root = ET.fromstring(melts.equilibrateResultsAsXML())
    xPlot[i] = t - 273.15
    for j in range (0, n):
        if root.find(".//System/Phase[@Type='" + plotPhases[j] + "']/Mass") != None:
            value = root.find(".//System/Phase[@Type='" + plotPhases[j] + "']/Mass").text
        else:
            value = 0.0
        yPlot[j][i] = value
    j = 0
    for (graph,) in graphs:
        graph.set_xdata(xPlot)
        graph.set_ydata(yPlot[j])
        j = j + 1
    fig.canvas.draw()
    updateExcelWorkbook(wb, root)

writeExcelWorkbook(wb, "MELTSv102summary.xlsx")